In [ ]:
import sys
import os
# Navigate up one level to the parent directory and append it to sys.path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
import nfl_data_py as nfl
import pandas as pd
from src import utils
from src import homers

# Add Latest Week

In [ ]:
SEASON, WEEK = 2023, 8

In [ ]:
new_week_raw = pd.read_clipboard()
print(new_week_raw.shape)
new_week_raw.head()

In [ ]:
new_week_processed = homers.process_picks(new_week_raw, WEEK, SEASON)
print(new_week_processed.shape)
new_week_processed.head()

In [ ]:
pick_df = pd.read_pickle(homers.PROCESSED_FILE_PATH)
pick_df.shape

In [ ]:
pick_df = pd.concat([pick_df, new_week_processed], ignore_index=True).drop_duplicates()
pick_df.shape

In [ ]:
pick_df.to_pickle(homers.PROCESSED_FILE_PATH)

# Add 2023 Data

In [ ]:
season = 2023
max_week = 4
nfl_df = nfl.import_schedules([season])[['game_id', 'season', 'week', 'away_team', 'home_team', 'away_score', 'home_score', 'result', 'spread_line']] 


weekly_picks = [processed]

for week in range(2, max_week + 1):
    df = pd.read_excel(f'{HOMERS_PATH}/homers-{season}.xlsx', sheet_name=f'Wk {week}')
    # df = df.dropna(axis=1, how='all')
    df = utils.clean_df_columns(df)
    df = df.rename(columns={'team': 'final', 'pick': 'pick_type'})
    assert all(col in ['pick_type', 'final', 'ben', 'chuck', 'griffin', 'harry', 'hunter'] for col in df.columns), print(df.columns)
    assert len(df) == 9
    df['pick_type'] = df.pick_type.map({i: 'reg' for i in range(1, 6)}).fillna(df['pick_type']).apply(str.lower)
    df['pick_type'] = df.pick_type.apply(lambda x: 'reg' if x not in ['bb', 'reg', 'sd', 'ud', 'mnf'] else x)
    # melt picks into a long table format
    transformed = pd.melt(df, id_vars=['pick_type'], var_name='picker', value_name='pick').dropna()
    transformed['season'] = season
    transformed['week'] = week
    transformed['pick'] = transformed['pick'].apply(str.upper).apply(str.strip).apply(lambda x: x.split('/')[0] if '/' in x else x)
    transformed = transformed[transformed['pick'] != '']
    # Map bad team names
    transformed['pick'] = transformed['pick'].apply(utils.standardize_teams)

    # turn pick types into one hot cols to make lookup faster
    transformed['spread_pick'] = transformed['pick_type'].map({'ud': False, 'sd': False}).fillna(True)
    transformed['best_bet'] = transformed['pick_type'] == 'bb'
    transformed['underdog_pick'] = transformed['pick_type'] == 'ud'
    transformed['survivor_pick'] = transformed['pick_type'] == 'sd'
    transformed['mnf_pick'] = transformed['pick_type'] == 'mnf'
    transformed = transformed.drop(columns='pick_type')

    # join home and away picks
    joined_away = pd.merge(
        transformed, 
        nfl_df,
        left_on=['season', 'week', 'pick'],
        right_on=['season', 'week', 'away_team']
    )
    joined_home = pd.merge(
        transformed, 
        nfl_df,
        left_on=['season', 'week', 'pick'],
        right_on=['season', 'week', 'home_team']
    )


    week_final_picks = pd.concat([joined_home, joined_away], ignore_index=True)
    week_final_picks['away_pick'] = week_final_picks['pick'] == week_final_picks['away_team']
    week_final_picks['away_cover'] = week_final_picks.apply(utils.cover_result, axis=1)
    week_final_picks['home_cover'] = 1 - week_final_picks['away_cover']
    week_final_picks['pick_result'] = week_final_picks.apply(utils.pick_result, axis=1)

    # add to list of picks
    weekly_picks.append(week_final_picks)
    print(f'{season} week {week}')
    del df

processed = pd.concat(weekly_picks, ignore_index=True)

In [ ]:
processed

In [ ]:
processed.to_pickle(f'{HOMERS_PATH}/homers-processed.pkl')

# Process Raw Picks

In [ ]:
seasons = [2021, 2022]

nfl_df = nfl.import_schedules(seasons)[['game_id', 'season', 'week', 'away_team', 'home_team', 'away_score', 'home_score', 'result', 'spread_line']] 
print('imported seasons')

season_picks = []

for season in seasons:
    weekly_picks = []

    for week in range(1, 19):
        df = pd.read_excel(f'{HOMERS_PATH}/homers-{season}.xlsx', sheet_name=f'Wk {week}', usecols=range(14), header=1)
        df = df.drop(df.index[9:])
        df = df.dropna(axis=1, how='all')
        if 'Notes' in df.columns:
            df = df.drop(columns=['Notes'])
        df = df.drop(columns=[col for col in df.columns if 'Unnamed' in col])
        df = utils.clean_df_columns(df)
        df = df.rename(columns={'team': 'final', 'picks': 'pick_type'})
        assert all(col in ['pick_type', 'final', 'ben', 'chuck', 'griffin', 'harry', 'hunter'] for col in df.columns)
        assert len(df) == 9
        df['pick_type'] = df.pick_type.map({i: 'reg' for i in range(1, 6)}).fillna(df['pick_type']).apply(str.lower)
        df['pick_type'] = df.pick_type.map({'underdog': 'ud', 'survivor': 'sd'}).fillna(df['pick_type'])
        df['pick_type'] = df.pick_type.apply(lambda x: 'reg' if x not in ['bb', 'reg', 'sd', 'ud', 'mnf'] else x)
        # melt picks into a long table format
        transformed = pd.melt(df, id_vars=['pick_type'], var_name='picker', value_name='pick').dropna()
        transformed['season'] = season
        transformed['week'] = week
        transformed['pick'] = transformed['pick'].apply(str.upper).apply(str.strip).apply(lambda x: x.split('/')[0] if '/' in x else x)
        transformed = transformed[transformed['pick'] != '']
        # Map bad team names
        transformed['pick'] = transformed['pick'].apply(utils.standardize_teams)

        # turn pick types into one hot cols to make lookup faster
        transformed['spread_pick'] = transformed['pick_type'].map({'ud': False, 'sd': False}).fillna(True)
        transformed['best_bet'] = transformed['pick_type'] == 'bb'
        transformed['underdog_pick'] = transformed['pick_type'] == 'ud'
        transformed['survivor_pick'] = transformed['pick_type'] == 'sd'
        transformed['mnf_pick'] = transformed['pick_type'] == 'mnf'
        transformed = transformed.drop(columns='pick_type')

        # join home and away picks
        joined_away = pd.merge(
            transformed, 
            nfl_df,
            left_on=['season', 'week', 'pick'],
            right_on=['season', 'week', 'away_team']
        )
        joined_home = pd.merge(
            transformed, 
            nfl_df,
            left_on=['season', 'week', 'pick'],
            right_on=['season', 'week', 'home_team']
        )


        week_final_picks = pd.concat([joined_home, joined_away], ignore_index=True)
        week_final_picks['away_pick'] = week_final_picks['pick'] == week_final_picks['away_team']
        week_final_picks['away_cover'] = week_final_picks.apply(utils.cover_result, axis=1)
        week_final_picks['home_cover'] = 1 - week_final_picks['away_cover']
        week_final_picks['pick_result'] = week_final_picks.apply(utils.pick_result, axis=1)

        # add to list of picks
        weekly_picks.append(week_final_picks)
        print(f'{season} week {week}')

    season_picks.append(pd.concat(weekly_picks, ignore_index=True))

df = pd.concat(season_picks, ignore_index=True)

In [ ]:
df.head()

In [ ]:
df.to_pickle(f'{HOMERS_PATH}/homers-processed.pkl')